In [ ]:
import torch
import math
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
import json
import tqdm
import operator
import random
import re
import pandas as pd

In [ ]:
HATESPEECH_LANGS = [
    "en",
    "es",
    "pt",
    "pl",
    "it"
]

language_dict = {
        "es": {
        "prompt": "La oración es  ",
        "neg1": "normal.",
        "neg2": "común.",
        "neg3": "ok.",
        "neg4": "usual.",
        "neg5": "aceptable.",
        "pos1": "sexista.",
        "pos2": "racista.",
        "pos3": "ofensiva.",
        "pos4": "abusivo.",
        "pos5": "odioso.",
    },
        "it": {
        "prompt": "La frase è ",
        "neg1": "normale.",
        "neg2": "comune.",
        "neg3": "ok.",
        "neg4": "solito.",
        "neg5": "accettabile.",
        "pos1": "sessista.",
        "pos2": "razzista.",
        "pos3": "offensivo.",
        "pos4": "abusivo.",
        "pos5": "odioso.",
    },
        "en": {
        "prompt": "The sentence is ",
        "neg1": "normal.",
        "neg2": "common.",
        "neg3": "ok.",
        "neg4": "usual.",
        "neg5": "acceptable.",
        "pos1": "sexist.",
        "pos2": "racist.",
        "pos3": "offensive.",
        "pos4": "abusive.",
        "pos5": "hateful.",
    },
        "pt": {
        "prompt": "A sentença é  ",
        "neg1": "normal.",
        "neg2": "comum.",
        "neg3": "ok.",
        "neg4": "habitual.",
        "neg5": "aceitável.",
        "pos1": "sexista.",
        "pos2": "racista.",
        "pos3": "ofensiva.",
        "pos4": "abusivo.",
        "pos5": "odioso.",
    },
        "pl":{
        "prompt": "Zdanie brzmi ",
        "neg1": "normaln.",
        "neg2": "pospolity.",
        "neg3": "ok.",
        "neg4": "zwykły.",
        "neg5": "do przyjęcia.",
        "pos1": "seksistowski.",
        "pos2": "rasistowski.",
        "pos3": "ofensywa.",
        "pos4": "obraźliwy.",
        "pos5": "nienawistny.",
    }
}

# PROMPT
# The sentence is <candidate>
# Candidates negative: normal., common., ok., usual., acceptable.
# Candidates positive: sexist., racist., offensive., abusive., hateful.

In [ ]:
def hatespeech_eval(prompt_n1, prompt_n2, prompt_n3, prompt_n4, prompt_n5, 
                    prompt_p1, prompt_p2, prompt_p3, prompt_p4, prompt_p5):
    ppl1, nll1, scores1 = get_logprobs(prompt_n1)
    ppl2, nll2, scores2 = get_logprobs(prompt_n2)
    ppl3, nll3, scores3 = get_logprobs(prompt_n3)
    ppl4, nll4, scores4 = get_logprobs(prompt_n4)
    ppl5, nll5, scores5 = get_logprobs(prompt_n5)
    
    ppl6, nll6, scores6 = get_logprobs(prompt_p1)
    ppl7, nll7, scores7 = get_logprobs(prompt_p2)
    ppl8, nll8, scores8 = get_logprobs(prompt_p3)
    ppl9, nll9, scores9 = get_logprobs(prompt_p4)
    ppl10, nll10, scores10 = get_logprobs(prompt_p5)
    
    sc_n1, sc_n2, sc_n3, sc_n4, sc_n5 = scores1.sum(), scores2.sum(), scores3.sum(), scores4.sum(), scores5.sum()
    sc_p1, sc_p2, sc_p3, sc_p4, sc_p5 = scores6.sum(), scores7.sum(), scores8.sum(), scores9.sum(), scores10.sum()
    my_list = [sc_n1, sc_n2, sc_n3, sc_n4, sc_n5, sc_p1, sc_p2, sc_p3, sc_p4, sc_p5]
    index, value = max(enumerate(my_list), key=operator.itemgetter(1))
    
        
    neg = [scores1.tolist(), scores2.tolist(), scores3.tolist(), scores4.tolist(), scores5.tolist()]
    pos = [scores6.tolist(), scores7.tolist(), scores8.tolist(), scores9.tolist(), scores10.tolist()]

    promts = [prompt_n1, prompt_n2, prompt_n3, prompt_n4, prompt_n5, prompt_p1, prompt_p2, prompt_p3, prompt_p4, prompt_p5]

        
    if index in [1,2,3,4,5]:
        predict = 0
    else:
        predict = 1
    return predict, my_list, neg, pos, promts


In [ ]:
prompt

In [ ]:
def get_logprobs(prompt):
    import re
    prompt = re.sub("\n+", "\n", prompt)
    
    nll = model.forward([prompt], loss_per_pos=False)[0][0]  
    ppl = math.exp(nll)
    losses = model.forward([prompt], loss_per_pos=True)[0][0]
    return (
        ppl,
        nll,
        np.array([loss * -1.0 for loss in losses]),
    )

## Evaluation

In [ ]:
from inference import load_mgpt

In [ ]:
model = load_mgpt("sberbank-ai/mGPT")

In [ ]:
with open("data/hate_speech_test.json") as inf:
    hatedata = json.load(inf)

In [ ]:
with open("data/hate_speech_train.json") as inf:
    hatedata_train = json.load(inf)

In [ ]:
for lang in language_dict:
    print(lang, len(hatedata[lang]))

In [ ]:
for lang in language_dict:
    print(lang, len(hatedata[lang]))

In [ ]:
def get_train_df(hatedata, lang, language_dict):

    label_dict = {0: ['neg1', 'neg2', 'neg3', 'neg4', 'neg5'],
                 1: ['pos1', 'pos2', 'pos3', 'pos4', 'pos5']}

    df = pd.DataFrame(hatedata[lang], columns=['label', 'text'])
    samples = []
    for row in df.values:
        labels = label_dict[row[0]]
        sample = np.random.choice(labels)
        prompt = "{}{} {}".format(
            language_dict[lang]["prompt"],
            language_dict[lang][sample],
            row[1]
        )
        samples.append(prompt)
    df['prompt'] = samples
    return df

In [ ]:
def preprocess_train_data(hatedata, lang, language_dict):
    label_dict = {0: ['neg1', 'neg2', 'neg3', 'neg4', 'neg5'],
                 1: ['pos1', 'pos2', 'pos3', 'pos4', 'pos5']}

    train_data = []
    for row in hatedata[lang]:
        labels = label_dict[row[0]]
        sample = np.random.choice(labels)
        prompt = "{}{} {}".format(
            language_dict[lang]["prompt"],
            language_dict[lang][sample],
            row[1]
        )
        train_data.append(prompt)
    return train_data

In [ ]:
LANGS = ['pt', 'pl', 'es', 'it', 'en']
num_shots = 0

In [ ]:
all_results = {}
for lang in LANGS:
    if lang not in all_results:
        all_results[lang] = {'gold':[], 'pred':[]}
    results_pred = []
    results_gold = []
    print(lang)

    train_data = preprocess_train_data(hatedata_train, lang, language_dict)
    
    scores = []
    neg_scores = []
    pos_scores = []
    promts_list = []
    for idx, row in enumerate(tqdm.tqdm(hatedata[lang])):

        if num_shots:
            sample = np.random.choice(train_data, num_shots)
            shots = '\n'.join(sample) + '\n'
        else:
            shots = ''
        
        propmpts_n = []
        for i in range(1,6):
            prompt = "{}{}{} {}".format(
                shots,
                language_dict[lang]["prompt"],
                language_dict[lang]["neg{}".format(i)],
                row[1]
            )
            propmpts_n.append(prompt)

        propmpts_p = []
        for i in range(1,6):
            prompt = "{}{}{} {}".format(
                shots,
                language_dict[lang]["prompt"],
                language_dict[lang]["pos{}".format(i)],
                row[1]
            )
            propmpts_p.append(prompt)

        prompt_n1, prompt_n2, prompt_n3, prompt_n4, prompt_n5 = propmpts_n
        prompt_p1, prompt_p2, prompt_p3, prompt_p4, prompt_p5 = propmpts_p

        label_gold = row[0]
        predict, my_list, neg, pos, promts = hatespeech_eval(
            prompt_n1, prompt_n2, prompt_n3, prompt_n4, prompt_n5, 
            prompt_p1, prompt_p2, prompt_p3, prompt_p4, prompt_p5
        )
        results_pred.append(str(predict))
        results_gold.append(str(label_gold))
    
        scores.append(my_list)
        neg_scores.append(neg)
        pos_scores.append(pos)
        promts_list.append(promts)
    
    all_results[lang]['gold'] = results_gold
    all_results[lang]['pred'] = results_pred

    accuracy = accuracy_score(results_gold, results_pred)
    pr_ma, rc_ma, f1_ma, _ = precision_recall_fscore_support(results_gold, results_pred, average='macro')
    pr_mi, rc_mi, f1_mi, _ = precision_recall_fscore_support(results_gold, results_pred, average='micro')
    pr_w, rc_w, f1_w, _ = precision_recall_fscore_support(results_gold, results_pred, average='weighted')
    print(lang)
    print(accuracy, len(results_gold))
    print(pr_ma, rc_ma, f1_ma)
    print(pr_mi, rc_mi, f1_mi)
    print(pr_w, rc_w, f1_w)
    print('--------------')

In [ ]:
for lang, vals in all_results.items():
    results_gold = vals['gold']
    results_pred = vals['pred']
    accuracy = accuracy_score(results_gold, results_pred)
    pr_ma, rc_ma, f1_ma, _ = precision_recall_fscore_support(results_gold, results_pred, average='macro')
    pr_mi, rc_mi, f1_mi, _ = precision_recall_fscore_support(results_gold, results_pred, average='micro')
    pr_w, rc_w, f1_w, _ = precision_recall_fscore_support(results_gold, results_pred, average='weighted')
    print(lang)
    print(accuracy, len(results_gold))
    print(pr_ma, rc_ma, f1_ma)
    print(pr_mi, rc_mi, f1_mi)
    print(pr_w, rc_w, f1_w)
    print('--------------')